In [1]:
import sys
import threading
import time
import zmq
import ceo
import numpy as np
from collections import OrderedDict
import os
import shelve
import traceback
import scipy.linalg as LA
import pickle
import zlib
import logging
import copy
from numpy.linalg import norm

from ruamel.yaml import YAML
yaml=YAML(typ='safe')

In [2]:
path_to_config_dir = './dos/trivial_AcO'

In [3]:
logger = logging.getLogger('Teste')
logger.setLevel(logging.INFO)

DOS_REPO = path_to_config_dir
cfg_file = os.path.join(path_to_config_dir,'dos.yaml')
logger.info('Reading config from %s',cfg_file)
with open(cfg_file) as f:
    cfg = yaml.load(f)

In [4]:
agent = None
N_SAMPLE = int(cfg['simulation']['sampling frequency']*
               cfg['simulation']['duration'])

In [5]:
from dos.dos import Logs

In [6]:
__k_step = 0
pushed = False
initialized = False
tau = 1/cfg['simulation']['sampling frequency']
logs = Logs(tau,logs_repo=DOS_REPO)

In [7]:
from dos.driver import Client, Atmosphere
verbose=logging.INFO

In [8]:
cfg['drivers'].keys()

dict_keys(['M1', 'M2', 'wfs48', 'AcO_controller', 'science'])

In [9]:
class IO:
    def __init__(self,tag,size=0, lien=None, logs=None):
        self.logger = logging.getLogger(tag)
        self.logger.setLevel(logging.INFO)
        self.size = size
        self.data = np.zeros(size)
        self.lien = lien
        self.logs = logs

In [10]:
class Input(IO):
    def __init__(self,*args,**kwargs):
        IO.__init__(self,*args,**kwargs)
    def tie(self,drivers):
        if self.lien is not None:
            d,io = self.lien
            self.logger.info('Linked to %s from %s',io,d)
            self.data = drivers[d].outputs[io].data
            self.size = self.data.shape

In [11]:
class Driver:
    def __init__(self,tau,tag):
        self.tau = tau
        self.tag = tag
        self.delay = 0
        self.sampling_rate = 1
    def start(self):
        pass
    def init(self):
        pass
    def update(self,_):
        pass
    def output(self,_):
        pass
    def terminate(self):
        pass

In [12]:
class Server(Driver):
    def __init__(self,tau,tag,logs,server,delay=0,sampling_rate=1,
             verbose=logging.INFO,**kwargs):
        Driver.__init__(self,tau,tag)
        self.logger = logging.getLogger(tag)
        self.logger.setLevel(verbose)
        self.delay         = delay
        self.sampling_rate = sampling_rate
        self.inputs       = {}
        try:
            for k in kwargs['inputs']:
                v = kwargs['inputs'][k]
                self.logger.info('New input: %s',k)
                self.inputs[k] = Input(k,**v)
        except KeyError:
            self.logger.info('No inputs!')
        self.outputs       = {}
        try:
            for k in kwargs['outputs']:
                v = kwargs['outputs'][k]
                self.logger.info('New output: %s',k)
                if not 'sampling_rate' in v:
                    v['sampling_rate']=self.sampling_rate
                if v['sampling_rate']<self.sampling_rate:
                    if v['sampling_rate']!=1:
                        self.logger.error('The driver output rate cannot be less than the update rate!')
                    self.logger.warning('Changing the output rate to match the update rate!')
                    v['sampling_rate'] = self.sampling_rate
                if 'logs' in v:
#                    logs.add(tag,k,v['logs']['decimation'],self.delay)
#                    if v['logs']['decimation']<v['sampling_rate']:
#                        if v['logs']['decimation']!=1:
#                            self.logger.error('The log decimation rate cannot be less than the output rate!')
#                        self.logger.warning('Changing the decimation rate to match the output rate!')
#                        v['logs']['decimation'] = v['sampling_rate']
#                    v['logs'] = logs.entries[tag][k]
                    self.logger.info('Output logged in!')
                self.outputs[k] = Output(k,**v)
        except KeyError:
            self.logger.info('No inputs!')
        try:
            self.shape = kwargs['shape']
        except KeyError:
            self.shape = None
        try:
            self.split = kwargs['split']
        except KeyError:
            self.split = {'indices_or_sections':1,'axis':0}
        self.server        = server
        self.msg = {'class_id':'',
                    'method_id':'',
                    'args':{}}
        self.msg_args = {'Start':{},
                    'Init':{},
                    'Update':{'inputs':{}},
                    'Outputs':{'outputs':[]},
                    'Terminate':{'args':None}}
        
    def associate(self,prm):
        base_units = np.pi/180
        units = {'degree': base_units,
                 'arcmin': base_units/60,
                 'arcsec': base_units/60/60,
                 'mas': base_units/60/60/1e3}
        if 'mirror' in prm:
            self.msg['class_id'] = 'GMT'
            self.msg_args['Start'].update(prm)
            if 'state' in prm:
                self.msg_args['Init']['state'] = {prm['mirror']:
                                                  {k:np.asarray(v,dtype=np.double) \
                                                   for k,v in prm['state'].items()}}
                self.msg_args['Start'].pop('state')
            self.msg_args['Update']['mirror'] = prm['mirror']
            self.msg_args['Update']['inputs'].update(\
                    {k_i:v_i.data for k_i,v_i in self.inputs.items()})

In [13]:
class Output(IO):
    def __init__(self,*args,sampling_rate=1,**kwargs):
        IO.__init__(self,*args,**kwargs)
        self.sampling_rate = sampling_rate
    def tie(self,drivers):
        if self.lien is not None:
            d,io = self.lien
            self.logger.info('Linked to %s from %s',io,d)
            self.data = drivers[d].inputs[io].data
            self.size = self.data.shape

In [14]:
drivers = {}
for d,v in cfg['drivers'].items():
    prm_file = os.path.join(path_to_config_dir,d)
    if os.path.isfile(prm_file+'.yaml') or os.path.isfile(prm_file+'.pickle'):
        logger.info('New driver: %s',d)
        if 'server' in v and v['server'] is False:
            drivers[d] = Client(tau,d,
                                            logs,
                                            verbose=verbose,**v)
        elif d=='atmosphere':
            drivers[d] = Atmosphere(tau,d,agent,
                                                    verbose=verbose)
        else:
            print(v)
            drivers[d] = Server(tau,d,logs,agent,verbose=verbose,**v)
    else:
        logger.warning('%s is missing!',prm_file)

INFO:Teste:New driver: M1
INFO:M1:New input: Txyz
INFO:M1:New input: Rxyz
INFO:M1:New input: modes
INFO:M1:No inputs!
INFO:Teste:New driver: M2
INFO:M2:New input: Txyz
INFO:M2:New input: Rxyz
INFO:M2:No inputs!
INFO:Teste:New driver: wfs48
INFO:wfs48:No inputs!
INFO:wfs48:New output: M12_RBM
INFO:wfs48:Output logged in!
INFO:Teste:New driver: AcO_controller
INFO:AcO_controller:New input: x_hat
INFO:AcO_controller:New output: M1 Txyz
INFO:AcO_controller:Output logged in!
INFO:AcO_controller:New output: M1 Rxyz
INFO:AcO_controller:Output logged in!
INFO:AcO_controller:New output: M2 Txyz
INFO:AcO_controller:Output logged in!
INFO:AcO_controller:New output: M2 Rxyz
INFO:AcO_controller:Output logged in!
INFO:AcO_controller:New output: M1 BM
INFO:AcO_controller:Output logged in!
INFO:Teste:New driver: science
INFO:science:No inputs!
INFO:science:New output: wfe_rms
INFO:science:Output logged in!
INFO:science:New output: segment_wfe_rms
INFO:science:Output logged in!
INFO:science:New output:

{'delay': 0, 'sampling_rate': 1, 'inputs': {'Txyz': {'size': [7, 3]}, 'Rxyz': {'size': [7, 3]}, 'modes': {'size': [7, 27]}}}
{'delay': 0, 'sampling_rate': 1, 'inputs': {'Txyz': {'size': [7, 3]}, 'Rxyz': {'size': [7, 3]}}}
{'delay': 0, 'sampling_rate': 1, 'outputs': {'M12_RBM': {'sampling_rate': 1, 'size': 273, 'logs': {'decimation': 1}}}}
{'outputs': {'wfe_rms': {'size': 1, 'logs': {'decimation': 1}}, 'segment_wfe_rms': {'size': 7, 'logs': {'decimation': 1}}, 'PSSn': {'size': 1, 'logs': {'decimation': 1}}, 'tiptilt': {'size': 2, 'logs': {'decimation': 1}}, 'segment_piston': {'size': 7, 'logs': {'decimation': 1}}, 'segment_tiptilt': {'size': 14, 'logs': {'decimation': 1}}}}


In [22]:
drivers['M1'].update('_')

In [23]:
 for k_d in drivers:
    d = drivers[k_d]
    for k_i in d.inputs:
        d.inputs[k_i].tie(drivers)
    for k_o in d.outputs:
        d.outputs[k_o].tie(drivers)


INFO:x_hat:Linked to M12_RBM from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


In [24]:
drivers['M1'].update('_')

In [25]:
for k_d in drivers:
    d = drivers[k_d]
    device = os.path.join(path_to_config_dir,k_d)
    try:
        with open(device+'.yaml') as f:
            prm = yaml.load(f)
    except:
        with open(device+'.pickle','rb') as f:
            prm = pickle.load(f)
    #print(prm)
    if 'mirror' in prm:
        print(k_d)
    #print('\n\n')
    d.associate(prm)

M1
M2


In [26]:
drivers['M1'].update('_')

In [29]:
__start = map(lambda x: x.start(), drivers.values())
__init = map(lambda x: x.init(), drivers.values())
#step = stepping()
__terminate = map(lambda x: x.terminate(), drivers.values())
logger.info('Simulation setup for a duration of {0}s @ {1}Hz ({2} steps)!'.format(
    cfg['simulation']['duration'],
    cfg['simulation']['sampling frequency'],
    N_SAMPLE))


INFO:Teste:Simulation setup for a duration of 50s @ 1Hz (50 steps)!


In [35]:
drivers['M1'].start()

In [36]:
drivers['M1'].init()

In [68]:
drivers['M1'].msg_args['Update']['inputs']['Txyz']

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [52]:
drivers['AcO_controller'].outputs['M1 Txyz'].data

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [31]:
list(__start)

[None, None, None, None, None]

In [32]:
list(__init)

[None, None, None, None, None]